In [1]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-

## File data.py
import re
import torch
import nltk
from nltk.corpus import wordnet as wn

## variables globales
vocab, vect_dict, embed_dict = [], {}, {}

def read_data(lang,type='vect'):
    vocab.clear()
    vect_dict.clear()
    embed_dict.clear()

    vectors = {}

    if lang == 'fra':
        if type == 'vect':
            file = "rg65_french.txt"
        else:
            file = "vecs100-linear-frwiki"
    else:
        if type == 'vect':
            file = "ws353.txt"
        else:
            file = "vecs100-linear-frwiki"

    with open(file, encoding='utf-8') as f:
        for line in f:
            l = line.split(" ")
            if (len(l) == 3):                                                   ## retrait du saut de ligne '\n' pour le fichier de similarité
                n = l[2].split('\n')
                l[2] = float(n[0])
                vector = [l[1],l[2]]
            else :
                del l[len(l)-1]
                vector = [float(val) for val in l[2:len(l)]]                    ## récupération des vecteurs du type [mot 2, valeur] ou du type [valeur1, valeur2,..., valeurX] pour les fichiers d'embeddings
            #print(len(vector))
            if len(vector) == 2:
              if l[0] not in vectors.keys():                                    ## ajout des vecteurs dans un dictionnaire avec le mot 1 en clé et le vecteur en valeur
                  vocab.append(l[0])
                  vectors[l[0]] = vector
                  if l[1] not in vectors.keys():                                ## ajout des vecteurs dans un dictionnaire avec le mot 2 en clé et le vecteur [mot 1, valeur] en valeur 
                    vocab.append(l[1])
                    vectors[l[1]] = [[l[0],float(l[2])]]
                  else :
                    vectors[l[1]].append([l[0],float(l[2])])
              else :
                  vectors[l[0]].append(vector)
                  if l[1] not in vectors.keys():                                ## ajout des vecteurs dans un dictionnaire avec le mot 2 en clé et le vecteur [mot 1, valeur] en valeur 
                    vectors[l[1]] = [[l[0],float(l[2])]]
                  else :
                    vectors[l[1]].append([l[0],float(l[2])])
            else :
              if l[0] not in vectors.keys():                                    ## ajout des vecteurs dans un dictionnaire avec le mot 1 en clé et le vecteur en valeur
                  vocab.append(l[0])
                  vectors[l[0]] = vector
              else :
                  vectors[l[0]].append(vector)
              
    return(vectors)



vect_dict = read_data("fra")                          ## dictionnaire pour la similarité cosinus
#print(vect_dict['corde'])
#print(vect_dict)
print("VECTS",len(vect_dict))

embed_dict = read_data("fra","embeds")                ## dictionnaire pour le retrofitting et la tâche d'analyse de sentiments
#print("corde : ", embed_dict['corde'])
#print("la : ", embed_dict['la'])
print("EMBEDS",len(embed_dict))

def find_vector(word,dic):
    ## Fonction qui permet de récupérer le vecteur d'un certain mot
    ## return [mot 2, value] pour le cas du dictionnaire vect_dict
    ## return [embeddings] pour le cas du dictionnaire embed_dict
    for key, value in dic.items():
        if word == key:
            return value
    return("Le mot n'a pas été trouvé dans le lexique.")

#print(find_vector("corde", embed_dict))
#print(find_vector("idk", embed_dict))

vocab = set(vocab)
print(len(vocab))

FileNotFoundError: [Errno 2] No such file or directory: 'rg65_french.txt'

In [8]:
## File retrofitting.py
nltk.download('omw')
nltk.download('wordnet')  # utilisation de WOLF via NLTK wordnet

#from data import vect_dict, embed_dict, vocab

DICT_NEIGBHORS = {}
DICT_HYPERNYMS = {}
DICT_HYPONYMS = {}

def get_synsets(word,lang):
    ## Méthode pour récupérer tous les mots en relation avec celui donnée en argument
    return wn.synsets(word,lang=lang)

#print("CHIEN",get_synsets('chien',lang='fra'))

def lemma(synsets,lang):
    ## Méthode pour récupérer uniquement les mots des synstes et pas 'word.n.01' par exemple
    lemmas,list_lemmas = [],[]
    for synset in synsets:
      lemmas = synset.lemma_names(lang)
      for lemma in lemmas:
        list_lemmas.append(lemma)
    return list_lemmas

#print("LEMMAS",lemma(get_synsets('chien','fra'),'fra'))

def get_hypernyms(word,lang):
    ## Méthode pour récupérer tous les mots en relation d'hypernymie avec celui donnée en argument
    synsets = wn.synsets(word,lang=lang)
    hyp = []
    if synsets != [] :
        for synset in synsets:
            hyp += synset.hypernyms()
    return hyp

#print("HYPERNYMS",get_hypernyms('chien','fra'))

def get_hyponyms(word,lang):
    ## Méthode pour récupérer tous les mots en relation d'hyponymie avec celui donnée en argument
    synsets = wn.synsets(word,lang=lang)
    hyp = []
    if synsets != [] :
        for synset in synsets:
            hyp += synset.hyponyms()
    return hyp

#print("HYPONYMS",get_hyponyms('chien','fra'))

def neighbors(word,lang,rel='neighb',dict_neighb={}):
    ## Méthode pour récupérer les voisins d'un certain mot en fonction du type de relation donnée en argument ou

    # Récupération des synsets avec le type de relation précisé ou non
    if rel == 'hyponym':
        synsets = get_hyponyms(word,lang)
    elif rel == 'hypernym':
        synsets = get_hypernyms(word,lang)
    else:
        synsets = get_synsets(word,lang)

    # Ajout de la liste de tous les mots appartenant au synset à un dictionnaire dont la clé est le mot donné en argument et la valeur est une liste de mots voisins
    if synsets != [] :
        if (word not in dict_neighb.keys()) or (dict_neighb == {}):
            dict_neighb[word] = list(set(lemma(synsets,lang)))
        else :
            dict_neighb[word].append(list(set(lemma(synsets,lang))))
    return dict_neighb


#print("NEIGHB",neighbors('chien','fra',DICT_NEIGBHORS))
#print("NEIGHB HYPONYM",neighbors('chien','fra','hyponym',DICT_HYPONYMS))
#print("NEIGHB HYPERNYM",neighbors('chien','fra','hypernym',DICT_HYPERNYMS))

def retrofit(num_iter,vocab,word_dict,lang,relation='neighb'):
  #vocab_size = len(vocab)
  vocabulary = vocab.intersection(set(word_dict.keys()))
  vectors_dict = word_dict

  for iter in range(num_iter):
    for word in vocabulary:
      sum = 0 

      if word in word_dict.keys(): 
        word_vect = word_dict[word]
        print("word vect",word_vect)
      else : word_vect = []
      list_neighb = neighbors(word,lang,relation)
      num_neighb = len(list_neighb)

      if num_neighb == 0: continue
      else :
        word_vect += word_dict[word]*num_neighb 
        for neighb in list_neighb:
          word_vect += vectors_dict[neighb]
          print("word vect 2",word_vect)
        vectors_dict[word] = [ word_vect[i]/(2*num_neighb) for i in range(len(word_vect)) ]

  return vectors_dict



print("TEST RETROFIT",retrofit(2,vocab,embed_dict,'fra'))

[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
word vect [0.179055, -0.205073, -0.51054, 0.011833, -0.206657, 0.302243, -0.266207, -0.12425, 0.049179, -0.142934, 0.002594, -0.537518, -0.008609, 0.419548, 0.568094, 0.298817, 0.307209, -0.173387, 0.361778, 0.87049, 0.232528, -0.275427, -0.138172, 0.338887, -0.141783, -0.070065, -0.042944, -0.743059, -0.087849, -0.54885, 0.406916, -0.201085, -0.033287, -0.100946, 0.037592, 0.266116, -0.214965, -0.335121, 0.089293, -0.05367, -0.021838, 0.175393, -0.368166, 0.39521, 0.512173, 0.077121, -0.263132, 0.43108, -0.019578, -0.13269, -0.847961, -0.003023, 0.143277, 0.267484, -0.146726, 0.052505, -0.470252, -0.050847, -0.173202, 0.96036, -0.651214, 0.151463, -1.199574, -0.217715, 0.345105, 0.161582, -0.255271, 0.152124, -0.276144, 0.042675, -0.026276, -0.230399, 0.021248, -0.73

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



NameError: ignored